In [1]:
# 모듈 import

from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
# import wandb
import matplotlib.pyplot as plt
import matplotlib.image as img

In [2]:
name = "MM_Faster_RCNN_Adam-0.00005_aug_768size"

classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('./configs/A_custom/cascade_swin_fpn_3x_ChangedIoU.py')

root='../../dataset/'

# dataset config 수정
cfg.data.train.classes = classes
cfg.data.train.img_prefix = root
cfg.data.train.ann_file = root + 'stratified_kfold/cv_train_1.json' # train json 정보
cfg.data.train.pipeline[2]['img_scale'] = (768,768) # Resize

cfg.data.val.classes = classes
cfg.data.val.img_prefix = root
cfg.data.val.ann_file = root + 'stratified_kfold/cv_val_1.json'
cfg.data.val.pipeline[1]["img_scale"] = (768,768)

cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json' # test json 정보
cfg.data.test.pipeline[1]['img_scale'] = (768,768) # Resize

cfg.data.samples_per_gpu = 4

cfg.seed = 2022
cfg.gpu_ids = [0]
cfg.work_dir = f"work_dirs/{name}"

cfg.model.roi_head.bbox_head.num_classes = 10

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=1)

cfg.evaluation = dict(interval=1, metric="bbox", save_best="bbox_mAP_50")


#============================custom============================
cfg.log_config.hooks[1]["init_kwargs"].name = name
cfg.optimizer = dict(type='Adam', lr=0.00005, weight_decay=0.0001)
cfg.runner.max_epochs=35

# build_dataset
datasets = [build_dataset(cfg.data.train)]

AttributeError: 'list' object has no attribute 'num_classes'

In [ ]:
# dataset 시각화
image = datasets[0][2389]["img"].data
image = image.permute(2,1,0)
plt.imshow(image)

# image.shape

In [394]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

2022-03-25 11:29:16,676 - mmcv - INFO - initialize ResNet with init_cfg {'type': 'Pretrained', 'checkpoint': 'torchvision://resnet50'}
2022-03-25 11:29:16,677 - mmcv - INFO - load model from: torchvision://resnet50
2022-03-25 11:29:16,677 - mmcv - INFO - load checkpoint from torchvision path: torchvision://resnet50
2022-03-25 11:29:16,879 - mmcv - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

2022-03-25 11:29:16,905 - mmcv - INFO - initialize FPN with init_cfg {'type': 'Xavier', 'layer': 'Conv2d', 'distribution': 'uniform'}
2022-03-25 11:29:16,932 - mmcv - INFO - initialize RPNHead with init_cfg {'type': 'Normal', 'layer': 'Conv2d', 'std': 0.01}
2022-03-25 11:29:16,940 - mmcv - INFO - initialize Shared2FCBBoxHead with init_cfg [{'type': 'Normal', 'std': 0.01, 'override': {'name': 'fc_cls'}}, {'type': 'Normal', 'std': 0.001, 'override': {'name': 'fc_reg'}}, {'type': 'Xavier', 'distribution': 'uniform', 'override'

In [395]:
# 모델 학습
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

2022-03-25 11:29:17,247 - mmdet - INFO - Start running, host: root@37aa03e6016b, work_dir: /opt/ml/detection/baseline/mmdetection/work_dirs/MM_Faster_RCNN_Adam-0.00005_aug_768size
2022-03-25 11:29:17,248 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTime

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


2022-03-25 11:29:43,906 - mmdet - INFO - Epoch [1][50/977]	lr: 4.945e-06, eta: 3:59:22, time: 0.421, data_time: 0.058, memory: 6411, loss_rpn_cls: 0.6133, loss_rpn_bbox: 0.0663, loss_cls: 1.3009, acc: 74.8262, loss_bbox: 0.0762, loss: 2.0567, grad_norm: 20.2568
2022-03-25 11:30:02,404 - mmdet - INFO - Epoch [1][100/977]	lr: 9.940e-06, eta: 3:44:37, time: 0.370, data_time: 0.007, memory: 6411, loss_rpn_cls: 0.2530, loss_rpn_bbox: 0.0723, loss_cls: 0.3216, acc: 94.9746, loss_bbox: 0.1744, loss: 0.8213, grad_norm: 7.8936
2022-03-25 11:30:21,217 - mmdet - INFO - Epoch [1][150/977]	lr: 1.494e-05, eta: 3:40:41, time: 0.376, data_time: 0.007, memory: 6411, loss_rpn_cls: 0.1801, loss_rpn_bbox: 0.0632, loss_cls: 0.3283, acc: 93.8164, loss_bbox: 0.2293, loss: 0.8010, grad_norm: 7.4925
2022-03-25 11:30:39,464 - mmdet - INFO - Epoch [1][200/977]	lr: 1.993e-05, eta: 3:36:58, time: 0.365, data_time: 0.007, memory: 6411, loss_rpn_cls: 0.1324, loss_rpn_bbox: 0.0464, loss_cls: 0.2811, acc: 94.6885, los

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 975/975, 24.6 task/s, elapsed: 40s, ETA:     0s

2022-03-25 11:36:07,048 - mmdet - INFO - Evaluating bbox...


...

Loading and preparing results...
DONE (t=0.04s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=3.84s).
Accumulating evaluation results...


2022-03-25 13:18:50,596 - mmdet - INFO - 
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.232
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.379
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.244
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.008
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.093
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.277
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.458
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.458
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.458
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.027
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.213
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000

DONE (t=1.03s).


2022-03-25 13:19:11,068 - mmdet - INFO - Epoch [17][50/977]	lr: 5.000e-07, eta: 1:51:08, time: 0.408, data_time: 0.059, memory: 6411, loss_rpn_cls: 0.0349, loss_rpn_bbox: 0.0296, loss_cls: 0.2513, acc: 92.3428, loss_bbox: 0.2125, loss: 0.5283, grad_norm: 3.5217
2022-03-25 13:19:30,112 - mmdet - INFO - Epoch [17][100/977]	lr: 5.000e-07, eta: 1:50:51, time: 0.381, data_time: 0.008, memory: 6411, loss_rpn_cls: 0.0325, loss_rpn_bbox: 0.0298, loss_cls: 0.2456, acc: 92.9082, loss_bbox: 0.1965, loss: 0.5045, grad_norm: 3.4717
2022-03-25 13:19:48,430 - mmdet - INFO - Epoch [17][150/977]	lr: 5.000e-07, eta: 1:50:33, time: 0.366, data_time: 0.008, memory: 6411, loss_rpn_cls: 0.0401, loss_rpn_bbox: 0.0321, loss_cls: 0.2435, acc: 92.8457, loss_bbox: 0.2071, loss: 0.5227, grad_norm: 3.3754
2022-03-25 13:20:07,317 - mmdet - INFO - Epoch [17][200/977]	lr: 5.000e-07, eta: 1:50:16, time: 0.378, data_time: 0.008, memory: 6411, loss_rpn_cls: 0.0316, loss_rpn_bbox: 0.0246, loss_cls: 0.2209, acc: 93.4688, 

KeyboardInterrupt: 